In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
sms_train = pd.read_csv('../sms_train.txt',delimiter='\t')
voice_train = pd.read_csv('../voice_train.txt',delimiter='\t')
sms_test = pd.read_csv('../sms_test_b.txt',delimiter='\t')
voice_test = pd.read_csv('../voice_test_b.txt',delimiter='\t')
sms_all = pd.concat([sms_train,sms_test])
voice_all = pd.concat([voice_train,voice_test])

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
f_train = open('../wa_train.txt',encoding='utf8')
df_train = []
for line in f_train.readlines():
    df_train.append(line.split('\t'))
df_train = pd.DataFrame(df_train)
df_train.columns = ['uid' ,'wa_name','cnt' ,'dura' ,'down','up' ,'type','day']
df_train.drop(0,inplace=True)

In [4]:
f_test = open('../wa_train.txt',encoding='utf8')
df_test = []
for line in f_test.readlines():
    df_test.append(line.split('\t'))
df_test = pd.DataFrame(df_test)
df_test.columns = ['uid' ,'wa_name','cnt' ,'dura' ,'down','up' ,'type','day']
df_test.drop(0,inplace=True)
wa_all= pd.concat([df_train,df_test])
wa_all['day'] = wa_all['day'].map(lambda x:str(x).strip('\n'))
wa_all.drop(wa_all[wa_all.day=='NULL'].index,axis=0,inplace=True)
wa_all = wa_all.drop_duplicates()
wa_all['day'] = wa_all['day'].map(lambda x:int(x))

In [24]:
sms_all.sort_values(['uid','start_time'],inplace=True)
voice_all.sort_values(['uid','start_time'],inplace=True)
wa_all.sort_values(['uid','day'],inplace=True)

In [26]:
import datetime
def time_fea(x):
    if int(x[0])>30:
        month = 5
        day = int(x[0])-30
        
    else:
        month = 4
        day = int(x[0])
    return '2017-%s-%s %s:%s:%s' % (month,day,int(x[1]),int(x[2]),int(x[3]))

In [27]:
sms_all['sec'] = sms_all['start_time'].map(lambda x:str(x)[-2:])
sms_all['min'] = sms_all['start_time'].map(lambda x:str(x)[-4:-2])
sms_all['hour'] = sms_all['start_time'].map(lambda x:str(x)[-6:-4])
sms_all['day'] = sms_all['start_time'].map(lambda x:str(x)[:-6])
sms_all['day'] = sms_all['day'].map(lambda x:1 if x=='' else int(x)+1)
sms_all['time_temp'] = list(zip(sms_all['day'].values,sms_all['hour'].values,sms_all['min'].values,sms_all['sec'].values))
sms_all['time'] = sms_all['time_temp'].map(time_fea)
sms_all['time'] = pd.to_datetime(sms_all['time'])
sms_all.drop(['sec','min','day','time_temp'],axis=1,inplace=True)
sms_all['date'] = sms_all['time'].dt.date

In [28]:
voice_all['s_sec'] = voice_all['start_time'].map(lambda x:str(x)[-2:])
voice_all['s_min'] = voice_all['start_time'].map(lambda x:str(x)[-4:-2])
voice_all['s_hour'] = voice_all['start_time'].map(lambda x:str(x)[-6:-4])
voice_all['s_day'] = voice_all['start_time'].map(lambda x:str(x)[:-6])
voice_all['s_day'] = voice_all['s_day'].map(lambda x:1 if x =='' else int(x)+1)
voice_all['time_temp'] = list(zip(voice_all['s_day'].values,voice_all['s_hour'].values,voice_all['s_min'].values,voice_all['s_sec'].values))
voice_all['s_time'] = voice_all['time_temp'].map(time_fea)
voice_all['s_time'] = pd.to_datetime(voice_all['s_time'])
voice_all.drop(['s_sec','s_min','s_day','time_temp'],axis=1,inplace=True)
voice_all['s_date'] = voice_all['s_time'].dt.date

In [29]:
voice_all['e_sec'] = voice_all['end_time'].map(lambda x:str(x)[-2:])
voice_all['e_min'] = voice_all['end_time'].map(lambda x:str(x)[-4:-2])
voice_all['e_hour'] = voice_all['end_time'].map(lambda x:str(x)[-6:-4])
voice_all['e_day'] = voice_all['end_time'].map(lambda x:str(x)[:-6])
voice_all['e_day'] = voice_all['e_day'].map(lambda x:1 if x =='' else int(x)+1)
voice_all['time_temp'] = list(zip(voice_all['e_day'].values,voice_all['e_hour'].values,voice_all['e_min'].values,voice_all['e_sec'].values))
voice_all['e_time'] = voice_all['time_temp'].map(time_fea)
voice_all['e_time'] = pd.to_datetime(voice_all['e_time'])
voice_all.drop(['e_sec','e_min','e_day','time_temp'],axis=1,inplace=True)
voice_all['e_date'] = voice_all['e_time'].dt.date
voice_all['dura'] = voice_all['e_time'] - voice_all['s_time']
voice_all['dura'] = voice_all['dura'].map(lambda x:x.total_seconds())

In [30]:
sms_opp_head_dict = {}
for i,va in enumerate(sms_all.opp_head.unique()):
    sms_opp_head_dict[va]=i
sms_opp_len_dict = {}
for i,va in enumerate(sms_all.opp_len.unique()):
    sms_opp_len_dict[va]=i
sms_hour_dict = {}
for i,va in enumerate(sms_all.hour.unique()):
    sms_hour_dict[va]=i

In [31]:
sms_fea = []

for uid in tqdm(sms_all['uid'].unique()):
    df = sms_all[sms_all.uid == uid]
    sms_cnt = len(df)
    max_sms = df.groupby('opp_num')['uid'].count().max() #对某一号码长度的短信数的最大值
    mean_sms = df.groupby('opp_num')['uid'].count().mean()#对某一个号码长度的短信数的平均值
    max_day_sms = df.groupby('date')['uid'].count().max() #对某一天的短信数的最大值
    mean_day_sms = df.groupby('date')['uid'].count().mean()#对某一天的短信数的平均值
    
    np_head = np.zeros(len(sms_opp_head_dict))
    sms_head = df.groupby('opp_head')['uid'].count()/sms_cnt
    sms_head.index = [sms_opp_head_dict[v] for v in sms_head.index]
    sms_head = sms_head.sort_index()
    np_head[sms_head.index] = sms_head #对某一特定开头的短信号码的短信数
    
    np_len = np.zeros(len(sms_opp_len_dict))
    sms_len = df.groupby('opp_len')['uid'].count()/sms_cnt
    sms_len.index = [sms_opp_len_dict[v] for v in sms_len.index]
    sms_len = sms_len.sort_index()
    np_len[sms_len.index] = sms_len#对某一特定长度的短信号码的短信数
    
    np_hour = np.zeros(len(sms_hour_dict))
    sms_hour = df.groupby('hour')['uid'].count()/sms_cnt
    sms_hour.index = [sms_hour_dict[v] for v in sms_hour.index]
    sms_hour = sms_hour.sort_index()
    np_hour[sms_hour.index] = sms_hour#在某一小时内的短息数
    
    df['dura'] = df['time'].diff().dt.total_seconds()
    avg_dura = df['dura'].dropna().mean() #每次发短信的间隔
    zero_dura = len(df[df.dura==0]) #发短信间隔为0的次数

    np_hour_dura = np.zeros(len(sms_hour_dict))
    sms_hour_dura = df.groupby('hour')['dura'].mean() 
    sms_hour_dura.index = [sms_hour_dict[v] for v in sms_hour_dura.index]
    sms_hour_dura = sms_hour_dura.sort_index()
    np_hour_dura[sms_hour_dura.index] = sms_hour_dura#每个时段的发短信的间隔

    inout = []
    for i in [0,1]:
        inout.append(len(df[df.in_out==i])/sms_cnt)
    
    sms_fea.append(np.hstack(([uid,sms_cnt,max_sms,mean_sms,max_day_sms,mean_day_sms,avg_dura,zero_dura],
                              np_head,np_len,np_hour,np_hour_dura,inout)))

  0%|                                                                                         | 0/7924 [00:00<?, ?it/s]D:\anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████████████████████████████████████████████████████████████████████████| 7924/7924 [10:55<00:00, 12.09it/s]


In [32]:
sms_df = pd.DataFrame(sms_fea)
sms_df.to_csv('sms_df_all.csv',index=False)

In [33]:
voice_opp_head_dict = {}
for i,v in enumerate(voice_all.opp_head.unique()):
    voice_opp_head_dict[v] = i
voice_opp_len_dict = {}
for i,v in enumerate(voice_all.opp_len.unique()):
    voice_opp_len_dict[v] = i
voice_opp_type_dict = {}
for i,v in enumerate(voice_all.call_type.unique()):
    voice_opp_type_dict[v] = i
voice_hour_dict = {}
for i,v in enumerate(voice_all.s_hour.unique()):
    voice_hour_dict[v] = i

In [34]:
voice_fea = []
for uid in tqdm(voice_all['uid'].unique()):
    df = voice_all[voice_all.uid==uid]
    voice_cnt = len(df)
    max_day = df.groupby('s_date')['uid'].count().max()#打电话最多的一天
    mean_day = df.groupby('s_date')['uid'].count().max()#平均每天打电话的数目
    max_num = df.groupby('opp_num')['uid'].count().max()#对某个号码打得最多的次数
    mean_num = df.groupby('opp_num')['uid'].count().mean()#对每个号码平均的打电话次数
    max_day_num = df.groupby(['opp_num','s_date'])['uid'].count().max()#某天对某个号码的最大打电话次数
    mean_day_num = df.groupby(['opp_num','s_date'])['uid'].count().mean()#某天对某个号码的平均电话次数
    
    np_head = np.zeros(len(voice_opp_head_dict))
    voice_head = df.groupby('opp_head')['uid'].count()/voice_cnt
    voice_head.index = [voice_opp_head_dict[v] for v in voice_head.index]
    voice_head = voice_head.sort_index()
    np_head[voice_head.index] = voice_head#对每种开头的号码的打电话次数
    
    np_len = np.zeros(len(voice_opp_len_dict))
    voice_len = df.groupby('opp_len')['uid'].count()/voice_cnt
    voice_len.index = [voice_opp_len_dict[v] for v in voice_len.index]
    voice_len = voice_len.sort_index()
    np_len[voice_len.index] = voice_len#对每种开头的号码的打电话次数
    
    np_type = np.zeros(len(voice_opp_type_dict))
    voice_type = df.groupby('call_type')['uid'].count()/voice_cnt
    voice_type.index = [voice_opp_type_dict[v] for v in voice_type.index]
    voice_type = voice_type.sort_index()
    np_type[voice_type.index] = voice_type#每种类型的打电话次数
    
    np_hour = np.zeros(len(voice_hour_dict))
    voice_hour = df.groupby('s_hour')['uid'].count()/voice_cnt
    voice_hour.index = [voice_hour_dict[v] for v in voice_hour.index]
    voice_hour = voice_hour.sort_index()
    np_hour[voice_hour.index] = voice_hour#每个小时的打电话次数
    
    np_type_dura = np.zeros(len(voice_opp_type_dict))
    voice_type_dura = df.groupby('call_type')['dura'].mean()
    voice_type_dura.index = [voice_opp_type_dict[v] for v in voice_type_dura.index]
    voice_type_dura = voice_type_dura.sort_index()
    np_type_dura[voice_type_dura.index] = voice_type_dura
    
    np_hour_dura = np.zeros(len(voice_hour_dict))
    voice_hour_dura = df.groupby('s_hour')['dura'].mean()
    voice_hour_dura.index = [voice_hour_dict[v] for v in voice_hour_dura.index]
    voice_hour_dura = voice_hour_dura.sort_index()
    np_hour_dura[voice_hour_dura.index] = voice_hour_dura
    
    in_cnt = len(df[df.in_out==1])
    out_cnt = len(df[df.in_out==0])
    
    in_dura = df[df.in_out==1]['dura'].mean()
    out_dura = df[df.in_out==0]['dura'].mean()
    
    
    voice_fea.append(np.hstack(([uid,max_day,mean_day,max_num,mean_num,max_day_num,mean_day_num],np_head,np_len,np_type,np_hour,
                              np_type_dura,np_hour_dura,[in_cnt,out_cnt,in_dura,out_dura] )))

100%|██████████████████████████████████████████████████████████████████████████████| 7979/7979 [22:52<00:00,  5.81it/s]


In [35]:
voice_df = pd.DataFrame(voice_fea)
voice_df.to_csv('voice_df_all.csv',index=False)

In [36]:
for uid in tqdm(wa_all['uid'].unique()[:10]):
    

,uid,wa_name,cnt,dura,down,up,type,day
8,u0001,360借条,10,11419,10535,3434,1,1
16,u0001,360安全中心网,109,105763,71003,127963,0,1
32,u0001,AccuWeather,6,16947,4509,15940,0,1
40,u0001,GPS定位服务器,2,4803,324,918,0,1
45,u0001,MIUI米柚,3,669,120,40,0,1
